## About the Smarta Byar Smart Village Platform in the Smart Village Platform

The Smart Village Platform is used by researchers to configure smart devices for improving traffic light configuration, and running on reports on traffic simulations configured at intersections in the world. 


## Deploy the Smarta Byar Smart Village Platform in the OpenShift Developer Sandbox

Before deploying the Smart Village platform, we are going to deploy the demo authentication secret connected to the Red Hat Single Sign-On server deployed by the Smart Village project in the New England Research Cloud at https://sso.smartabyarsmartvillage.org. This will allow GitHub users to connect to their own Smart Village applications deployed in their own OpenShift Sandbox and access all the data in their own sandbox application, and some minimal public data from the main Smart Village site deployed in the New England Research Cloud at https://www.smartabyarsmartvillage.org. 

In [ ]:
%%bash
oc apply -f ~/smartvillage-operator/static/demo-auth-secret.yaml
echo DONE

If you are curious what the SmartaByarSmartVillage custom resource definition looks like that you are deploying in the Sandbox, run the command below. 

In [ ]:
%%bash
cat ~/smartvillage-operator/kustomize/overlays/sandbox/smartabyarsmartvillages/smartvillage/smartabyarsmartvillage.yaml
echo DONE

Here are some useful things to note about the configuration of the IoT Agent JSON. 

- `auth:` We configure the authentication to our Red Hat Single Sign-On Server, to register OAuth2 / OpenID Connect authentication and authorization in the application which is integrated with GitHub. 
- `ngsi_ld:` We configure an NGSI-LD to Smart Village Sync application that integrates with the context broker. This service subscribes to IoT Edge device data changes in the context broker. 
- `kafka:` We disable Kafka in the Smart Village application because running Kafka requires too many resources that we don't have in a Developer Sandbox. 
- `zookeeper:` We will disable Zookeeper cluster manager because with all the starting and stopping of containers in the sandbox, the applications using Zookeeper as a cluster manager can start throwing errors when being restarted or having the cluster manager go down. 
- `postgres:` We configure the host to the PostgreSQL service for persistence in the Smarta Byar Smart Village Platform. 
- `solr:` We configure the host to the Solr service for serving up API data in the Smarta Byar Smart Village Platform. 
- The `site:` definition defines the memory and CPU requests and limits for our Smart Village pod. We have to keep this lower than I would recommend for a production deployment because we have limited resources available in the Developer Sandbox for the many running Smart Village services. 
- `import_data:` The Smart Village Platform can import data from various sources, but for this Developer Sandbox configuration, we disable data import. 
- `auth_role:` Each of the APIs in the Smart Village Platform require some kind of role-based access control. We have configured our Red Hat Single Sign-On Server `smartvillage-public` client to give all users the role `Public`. This means that any GitHub user who accesses their own application running in their Sandbox will be able to access the API. 

For more information about the SmartaByarSmartVillage custom resource definition, [see the full SmartaByarSmartVillage schema here](https://github.com/smartabyar-smartvillage/smartvillage-operator/blob/main/config/crd/bases/smartvillage.computate.org_smartabyarsmartvillages.yaml). 

Run the Ansible Playbook below to deploy the Smart Village Platform to the cloud. 


In [ ]:
%%bash
ansible-playbook ~/smartvillage-operator/apply-smartabyarsmartvillage.yaml \
  -e ansible_operator_meta_namespace=$(cat /var/run/secrets/kubernetes.io/serviceaccount/namespace) \
  -e crd_path=~/smartvillage-operator/kustomize/overlays/sandbox/smartabyarsmartvillages/smartvillage/smartabyarsmartvillage.yaml
echo DONE

### View Smart Village pod details
After running the Ansible Playbook, it will take a minute before the Smart Village pod is up and running. Run the command below until the Smart Village pod health checks are `READY 1/1` and `STATUS Running`. 

In [ ]:
%%bash
oc get pod -l app=smartabyar-smartvillage-web
oc wait pod -l app=smartabyar-smartvillage-web --for=condition=Ready --timeout=5m
oc get pod -l app=smartabyar-smartvillage-web
echo DONE

### View Smarta Byar Smart Village pod logs
If your Smarta Byar Smart Village pod does not reach the STATUS Running, you can run the command below to view the pod logs of Smarta Byar Smart Village and check for other errors that may have occured. 

In [ ]:
%%bash
oc logs -l app=smartabyar-smartvillage-web
echo DONE

## Next...
I hope that answers your questions about IoT Agent JSON in the Smart Village Platform. 
- If you have additional questions or issues, please [create an issue for the course here](https://github.com/smartabyar-smartvillage/smartabyar-smartvillage-sandbox-course/issues). 
- Otherwise, please continue to the next notebook [11-deploy-smart-devices.ipynb](11-deploy-traffic-flow-observed-devices.ipynb). 